In [2]:
!pip install datasets
!pip install -q emoji pythainlp sklearn-pycrfsuite seqeval
!pip install tensorflow deepcut
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from pythainlp.tokenize import word_tokenize
from datasets import load_dataset, load_metric, DatasetDict, Dataset, load_from_disk

In [5]:
dataset = load_from_disk('/content/drive/MyDrive/Fake news/News-Dataset/dataset')

In [6]:
def tokenizer(text):
  return word_tokenize(text,engine="deepcut")

In [7]:
tokenizer(dataset['train']['text'][0])

['วิธี',
 'ทำ',
 'หัวปลีกล้วยน้ำว้า',
 'กับ',
 'สูตร',
 'แก้',
 'โรค',
 'เบาหวาน',
 '|',
 '|',
 '|',
 'วิธี',
 'ทำ',
 'หัวปลีกล้วยน้ำว้า',
 'กับ',
 'สูตร',
 'แก้',
 'โรค',
 'เบาหวาน',
 'หัว',
 'ปลีกล้วยน้ำว้า',
 'กับ',
 'สูตร',
 'แก้',
 'โรค',
 'เบาหวาน',
 'มี',
 'วิธี',
 'ทำ',
 'รับประทาน',
 'เอง',
 'แบบ',
 'ง่าย',
 'ๆ',
 'คือ',
 'ให้',
 'เอา',
 'หัว',
 'ปลีกล้วยน้ำ',
 'ว้า',
 'เท่า',
 'นั้น',
 'หัว',
 'ปลี',
 'จาก',
 'กล้วย',
 'ชนิด',
 'อื่น',
 'ใช้',
 'ไม่',
 'ได้',
 'จำนวน',
 '1',
 'หัวย่างไฟ',
 'ให้',
 'เปลือก',
 'ชั้น',
 'นอก',
 'ไหม้',
 'เกรียม',
 'แล้ว',
 'ต้ม',
 'กับ',
 'น้ำ',
 'กะ',
 'จำนวน',
 'พอเหมาะ',
 'จน',
 'เดือด',
 'ดื่ม',
 'ต่าง',
 'น้ำ',
 'ทั้ง',
 'วัน',
 'ให้',
 'หมด',
 'หม้อ',
 'วัน',
 'ละ',
 '1',
 'หัว',
 'จน',
 'ครบ',
 '7',
 'วัน',
 'หาก',
 'คน',
 'ที่',
 'เป็น',
 'เบาหวาน',
 'ดื่ม',
 'แล้ว',
 'ถูก',
 'ทาง',
 'ยา',
 'ดื่ม',
 '1',
 '-',
 '2',
 'วัน',
 'แรก',
 'จะ',
 'มี',
 'อาการ',
 'รู้สึก',
 'ปวด',
 'เมื่อย',
 'ตาม',
 'ร่างกาย',
 'แทบ',
 'ทน',
 'ไม่',
 'ไหว',
 '

In [8]:
#เปลี่ยนข้อความเป็น bag-of-words features
tfidf = TfidfVectorizer(ngram_range=(1,2), tokenizer=word_tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1)

In [9]:
x_train = tfidf.fit_transform(dataset['train']['text']+dataset['valid']['text'])
x_test = tfidf.transform(dataset['test']['text'])
x_train,x_test

(<2700x74219 sparse matrix of type '<class 'numpy.float64'>'
 	with 1352829 stored elements in Compressed Sparse Row format>,
 <676x74219 sparse matrix of type '<class 'numpy.float64'>'
 	with 318216 stored elements in Compressed Sparse Row format>)

In [10]:
enc = LabelEncoder()
y_train = enc.fit_transform(np.array(dataset['train']['labels'] + dataset['valid']['labels']))
y_valid = enc.transform(np.array(dataset['valid']['labels']))
y_test = enc.transform(np.array(dataset['test']['labels']))
y_train.shape, y_valid.shape, y_test.shape

((2700,), (270,), (676,))

In [11]:
# https://towardsdatascience.com/quickly-test-multiple-models-a98477476f0

In [15]:
from skopt import BayesSearchCV
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.utils import class_weight
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
import pandas as pd

dfs = []
opt = BayesSearchCV(
    SVC(),
    {
        'C': (1e-6, 1e+6, 'log-uniform'),
        'gamma': (1e-6, 1e+1, 'log-uniform'),
        'degree': (1, 8),  # integer valued parameter
        'kernel': ['linear', 'poly', 'rbf'],  # categorical parameter
    },
    n_iter=32,
    cv=3
)
opt.fit(x_train, y_train)
target_names = ['true', 'fake']
print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(x_test, y_test))

val. score: 0.9077777777777777
test score: 0.908284023668639
